## Binance API Exploration

In [1]:
import numpy as np
import pandas as pd
import ast
import boto3
from binance.client import Client
# Instantiate resources
try:
    boto_session = boto3.Session(profile_name='loidsig')
except:
    boto_session = boto3.Session()
sm_client = boto_session.client(
    service_name='secretsmanager',
    region_name='us-east-1',
    endpoint_url='https://secretsmanager.us-east-1.amazonaws.com'
)
get_secret_value_response = sm_client.get_secret_value(SecretId='Loidsig_CPM_Binance')
key, value = ast.literal_eval(get_secret_value_response['SecretString']).popitem()
bnb_client = Client(key, value)

In [2]:
bnb_client.get_asset_balance(asset='eth')

{'asset': 'ETH', 'free': '1.02413568', 'locked': '0.00000000'}

In [12]:
order = bnb_client.order_market_buy(
    symbol='ETHUSDT',
    quantity=1
    ,newOrderRespType='FULL')
order

{'symbol': 'ETHUSDT',
 'orderId': 88463425,
 'clientOrderId': 'etZU82OIwTheMPBiVqdfBB',
 'transactTime': 1530932508055,
 'price': '0.00000000',
 'origQty': '1.00000000',
 'executedQty': '1.00000000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'fills': [{'price': '466.34000000',
   'qty': '1.00000000',
   'commission': '0.01705046',
   'commissionAsset': 'BNB',
   'tradeId': 30642842}]}

In [13]:
bnb_client.get_asset_balance(asset='usdt')

{'asset': 'USDT', 'free': '561.63554540', 'locked': '0.00000000'}

In [14]:
order = bnb_client.order_market_sell(
    symbol='ETHUSDT',
    quantity=1
    ,newOrderRespType='FULL')
order

{'symbol': 'ETHUSDT',
 'orderId': 88463720,
 'clientOrderId': 'SKahuOJuyTa2Y7MwnrqNSE',
 'transactTime': 1530932531224,
 'price': '0.00000000',
 'origQty': '1.00000000',
 'executedQty': '1.00000000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '465.68000000',
   'qty': '0.55655000',
   'commission': '0.00948620',
   'commissionAsset': 'BNB',
   'tradeId': 30643004},
  {'price': '465.67000000',
   'qty': '0.44345000',
   'commission': '0.00755829',
   'commissionAsset': 'BNB',
   'tradeId': 30643005}]}

In [45]:
fills_df = pd.DataFrame(order['fills'])
fills_df['commission'] = pd.to_numeric(fills_df['commission'], errors='coerce')
fills_df['price'] = pd.to_numeric(fills_df['price'], errors='coerce')
fills_df['qty'] = pd.to_numeric(fills_df['qty'], errors='coerce')
fills_df

,commission,commissionAsset,price,qty,tradeId
0,0.009486,BNB,465.68,0.55655,30643004
1,0.007558,BNB,465.67,0.44345,30643005


In [47]:
total = fills_df['commission'].sum()
total

0.01704449

In [52]:
fills_df.iloc[0]['commissionAsset']

'BNB'

In [48]:
total_qty = fills_df['qty'].sum()
total_qty

1.0

In [49]:
fills_df['qty_perc'] = fills_df['qty'] / total_qty
fills_df['price_x_qty_perc'] = fills_df['price'] * fills_df['qty_perc']
avg_price = fills_df['price_x_qty_perc'].sum()
avg_price

465.67556549999995

In [50]:
fills_df

,commission,commissionAsset,price,qty,tradeId,qty_perc,price_x_qty_perc
0,0.009486,BNB,465.68,0.55655,30643004,0.55655,259.174204
1,0.007558,BNB,465.67,0.44345,30643005,0.44345,206.501362


In [42]:
order

{'symbol': 'ETHUSDT',
 'orderId': 58986415,
 'clientOrderId': 'z2MXz9nxyAGU7C2vYLmGu3',
 'transactTime': 1524365564648,
 'price': '0.00000000',
 'origQty': '0.49000000',
 'executedQty': '0.49000000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL'}

In [6]:
recents_df = pd.DataFrame(client.get_aggregate_trades(symbol='BNBBTC'))
recents_df

,M,T,a,f,l,m,p,q
0,True,1524275411670,13613516,15794095,15794095,False,0.00141860,0.02000000
1,True,1524275417315,13613517,15794096,15794096,False,0.00141860,2.60000000
2,True,1524275419019,13613518,15794097,15794097,False,0.00141760,169.04000000
3,True,1524275419037,13613519,15794098,15794098,True,0.00141690,331.48000000
4,True,1524275419163,13613520,15794099,15794099,False,0.00141840,1.00000000
5,True,1524275421469,13613521,15794100,15794100,False,0.00141830,0.02000000
6,True,1524275421699,13613522,15794101,15794101,False,0.00141830,0.01000000
7,True,1524275421818,13613523,15794102,15794102,False,0.00141830,0.01000000
8,True,1524275426556,13613524,15794103,15794103,False,0.00141800,0.02000000
9,True,1524275428120,13613525,15794104,15794104,False,0.00141780,10.00000000


In [32]:
agg_trades = bnb_client.aggregate_trade_iter(symbol='ETHBTC', start_str='2018-04-21')#'1 Jan. 2018')
agg_trade_list = list(agg_trades)

df = pd.DataFrame(agg_trade_list)
df.rename(columns={'a':'trade_id','p':'price','q':'quantity','f':'first_trade_id','l':'last_trade_id',
                   'T':'micro_timestamp','m':'buyer_maker','M':'best_price_match'}, inplace=True)
df['unix_timestamp'] = df['micro_timestamp']/1000
df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='s')
df = df[['trade_id','datetime','price','quantity','buyer_maker','best_price_match',
         'first_trade_id','last_trade_id','micro_timestamp']]
df

,trade_id,datetime,price,quantity,buyer_maker,best_price_match,first_trade_id,last_trade_id,micro_timestamp
0,47965330,2018-04-21 00:00:00.184,0.06958100,0.33000000,True,True,54412267,54412270,1524268800184
1,47965331,2018-04-21 00:00:00.184,0.06957800,0.15000000,True,True,54412271,54412271,1524268800184
2,47965332,2018-04-21 00:00:00.184,0.06957500,0.25000000,True,True,54412272,54412274,1524268800184
3,47965333,2018-04-21 00:00:00.184,0.06957200,0.15000000,True,True,54412275,54412275,1524268800184
4,47965334,2018-04-21 00:00:00.184,0.06955600,0.01500000,True,True,54412276,54412276,1524268800184
5,47965335,2018-04-21 00:00:00.184,0.06955400,0.07500000,True,True,54412277,54412277,1524268800184
6,47965336,2018-04-21 00:00:00.856,0.06962400,0.01500000,False,True,54412278,54412278,1524268800856
7,47965337,2018-04-21 00:00:00.856,0.06965600,0.09600000,False,True,54412279,54412279,1524268800856
8,47965338,2018-04-21 00:00:00.989,0.06965600,0.01800000,False,True,54412280,54412280,1524268800989
9,47965339,2018-04-21 00:00:02.118,0.06965600,0.02300000,False,True,54412281,54412281,1524268802118


In [11]:
# Iterate over generator and identify days
agg_trades = bnb_client.aggregate_trade_iter(symbol='ETHBTC', start_str='1 day ago')
trade_dict_list = [next(agg_trades)]
trade_dict_list_ = []
day_id = int(trade_dict_list[0]['T']/1000/60/60/24)
for trade in agg_trades:
    if int(trade['T']/1000/60/60/24) > day_id:
        trade_dict_list_.append(trade)
        #print(f"End of day with {len(trade_dict_list)} trades.")
        #break
    else:
        trade_dict_list.append(trade)

df = pd.DataFrame(trade_dict_list)
df.rename(columns={'a':'trade_id','p':'price','q':'quantity','f':'first_trade_id','l':'last_trade_id',
                   'T':'micro_timestamp','m':'buyer_maker','M':'best_price_match'}, inplace=True)
df['unix_timestamp'] = df['micro_timestamp']/1000
df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='s')
df = df[['trade_id','datetime','price','quantity','buyer_maker','best_price_match',
         'first_trade_id','last_trade_id','micro_timestamp']]
df

,trade_id,datetime,price,quantity,buyer_maker,best_price_match,first_trade_id,last_trade_id,micro_timestamp
0,47786063,2018-04-20 09:47:19.485,0.06887000,0.00100000,True,True,54213896,54213896,1524217639485
1,47786064,2018-04-20 09:47:19.485,0.06886900,0.00100000,True,True,54213897,54213897,1524217639485
2,47786065,2018-04-20 09:47:19.512,0.06893800,2.56000000,False,True,54213898,54213898,1524217639512
3,47786066,2018-04-20 09:47:20.091,0.06887200,0.22400000,True,True,54213899,54213899,1524217640091
4,47786067,2018-04-20 09:47:20.283,0.06887200,0.13800000,True,True,54213900,54213900,1524217640283
5,47786068,2018-04-20 09:47:21.166,0.06887300,0.03100000,True,True,54213901,54213901,1524217641166
6,47786069,2018-04-20 09:47:22.228,0.06887400,1.29800000,True,True,54213902,54213902,1524217642228
7,47786070,2018-04-20 09:47:22.991,0.06887500,0.11800000,True,True,54213903,54213903,1524217642991
8,47786071,2018-04-20 09:47:23.254,0.06893800,0.13700000,False,True,54213904,54213904,1524217643254
9,47786072,2018-04-20 09:47:27.801,0.06887400,1.46000000,True,True,54213905,54213906,1524217647801


In [13]:
df = pd.DataFrame(trade_dict_list_)
df.rename(columns={'a':'trade_id','p':'price','q':'quantity','f':'first_trade_id','l':'last_trade_id',
                   'T':'micro_timestamp','m':'buyer_maker','M':'best_price_match'}, inplace=True)
df['unix_timestamp'] = df['micro_timestamp']/1000
df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='s')
df = df[['trade_id','datetime','price','quantity','buyer_maker','best_price_match',
         'first_trade_id','last_trade_id','micro_timestamp']]
df

,trade_id,datetime,price,quantity,buyer_maker,best_price_match,first_trade_id,last_trade_id,micro_timestamp
0,47965330,2018-04-21 00:00:00.184,0.06958100,0.33000000,True,True,54412267,54412270,1524268800184
1,47965331,2018-04-21 00:00:00.184,0.06957800,0.15000000,True,True,54412271,54412271,1524268800184
2,47965332,2018-04-21 00:00:00.184,0.06957500,0.25000000,True,True,54412272,54412274,1524268800184
3,47965333,2018-04-21 00:00:00.184,0.06957200,0.15000000,True,True,54412275,54412275,1524268800184
4,47965334,2018-04-21 00:00:00.184,0.06955600,0.01500000,True,True,54412276,54412276,1524268800184
5,47965335,2018-04-21 00:00:00.184,0.06955400,0.07500000,True,True,54412277,54412277,1524268800184
6,47965336,2018-04-21 00:00:00.856,0.06962400,0.01500000,False,True,54412278,54412278,1524268800856
7,47965337,2018-04-21 00:00:00.856,0.06965600,0.09600000,False,True,54412279,54412279,1524268800856
8,47965338,2018-04-21 00:00:00.989,0.06965600,0.01800000,False,True,54412280,54412280,1524268800989
9,47965339,2018-04-21 00:00:02.118,0.06965600,0.02300000,False,True,54412281,54412281,1524268802118


In [80]:
import boto3
boto_session = boto3.Session(profile_name='loidsig')
s3_resource = boto_session.resource('s3')
s3_bucket = 'loidsig-crypto'
my_bucket = s3_resource.Bucket(s3_bucket)
objects = my_bucket.objects.filter(Prefix='binance/historic_trades/ethusdt')
dates
for obj in objects:
    print(obj.key)


binance/historic_trades/ethusdt/2018-04-21.csv
binance/historic_trades/ethusdt/2018-04-22.csv


In [19]:
import time
int(time.time()/60/60/24)

17642

In [23]:
import dateparser
start_date = '1 Jan. 2018 UTC'
int(dateparser.parse(start_date).timestamp()/60/60/24)

17532

In [31]:
import datetime
datetime.datetime.utcfromtimestamp(int(dateparser.parse(start_date).timestamp()/60/60/24)*24*60*60).strftime('%Y-%m-%d')

'2018-01-01'

In [58]:
client.get_exchange_info()

{'exchangeFilters': [],
 'rateLimits': [{'interval': 'MINUTE',
   'limit': 1200,
   'rateLimitType': 'REQUESTS'},
  {'interval': 'SECOND', 'limit': 10, 'rateLimitType': 'ORDERS'},
  {'interval': 'DAY', 'limit': 100000, 'rateLimitType': 'ORDERS'}],
 'serverTime': 1521631358187,
 'symbols': [{'baseAsset': 'ETH',
   'baseAssetPrecision': 8,
   'filters': [{'filterType': 'PRICE_FILTER',
     'maxPrice': '100000.00000000',
     'minPrice': '0.00000100',
     'tickSize': '0.00000100'},
    {'filterType': 'LOT_SIZE',
     'maxQty': '100000.00000000',
     'minQty': '0.00100000',
     'stepSize': '0.00100000'},
    {'filterType': 'MIN_NOTIONAL', 'minNotional': '0.00100000'}],
   'icebergAllowed': True,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'quoteAsset': 'BTC',
   'quotePrecision': 8,
   'status': 'TRADING',
   'symbol': 'ETHBTC'},
  {'baseAsset': 'LTC',
   'baseAssetPrecision': 8,
   'filters': [{'filterType': 'PRICE_FIL

In [14]:
depth = client.get_order_book(symbol='BNBBTC')
depth

{'asks': [['0.00090240', '6.46000000', []],
  ['0.00090270', '551.72000000', []],
  ['0.00090280', '328.01000000', []],
  ['0.00090340', '24.00000000', []],
  ['0.00090350', '55.56000000', []],
  ['0.00090380', '1.77000000', []],
  ['0.00090430', '1.11000000', []],
  ['0.00090440', '9.00000000', []],
  ['0.00090450', '10.92000000', []],
  ['0.00090500', '75.10000000', []],
  ['0.00090510', '43.20000000', []],
  ['0.00090560', '337.28000000', []],
  ['0.00090580', '14.41000000', []],
  ['0.00090600', '500.67000000', []],
  ['0.00090620', '49.00000000', []],
  ['0.00090650', '9.00000000', []],
  ['0.00090660', '49.64000000', []],
  ['0.00090670', '10.00000000', []],
  ['0.00090680', '26.63000000', []],
  ['0.00090710', '16.10000000', []],
  ['0.00090740', '133.45000000', []],
  ['0.00090750', '8.02000000', []],
  ['0.00090760', '2.35000000', []],
  ['0.00090770', '194.70000000', []],
  ['0.00090780', '199.99000000', []],
  ['0.00090790', '3.00000000', []],
  ['0.00090800', '30.88000000',

In [17]:
import requests
import pandas as pd
response = requests.get('https://api.coinmarketcap.com/v1/ticker/')
coins_json = response.json()
coins_df = pd.DataFrame(coins_json)
coins_df

,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,6940240000.0,16905262.0,bitcoin,1520426366,179978490831,21000000.0,Bitcoin,0.15,-3.05,0.11,1.0,10646.3,1,BTC,16905262.0
1,1863960000.0,98038679.0,ethereum,1520426353,77540359987.0,None,Ethereum,0.34,-5.03,-9.18,0.0745427,790.916,2,ETH,98038679.0
2,710456000.0,39091956706.0,ripple,1520426341,35979064194.0,100000000000,Ripple,0.49,-2.15,-0.29,0.00008674,0.92037,3,XRP,99992520283.0
3,438627000.0,17004600.0,bitcoin-cash,1520426352,19978194402.0,21000000.0,Bitcoin Cash,0.82,-3.27,-4.11,0.11073,1174.87,4,BCH,17004600.0
4,656003000.0,55521831.0,litecoin,1520426341,10746861087.0,84000000.0,Litecoin,0.89,-4.21,-8.36,0.0182429,193.561,5,LTC,55521831.0
5,129638000.0,25927070538.0,cardano,1520426353,7078038403.0,45000000000.0,Cardano,1.33,-5.95,-14.23,0.00002573,0.272998,6,ADA,31112483745.0
6,231378000.0,65000000.0,neo,1520426349,7065760000.0,100000000.0,NEO,0.95,1.98,-20.12,0.0102452,108.704,7,NEO,100000000.0
7,39353500.0,18468387282.0,stellar,1520426343,6316742502.0,None,Stellar,1.72,-1.89,-1.09,0.00003224,0.34203,8,XLM,103748462367
8,102893000.0,15797832.0,monero,1520426342,5641642827.0,None,Monero,2.73,0.6,21.77,0.0336576,357.115,9,XMR,15797832.0
9,347984000.0,712657407.0,eos,1520426352,5065347926.0,1000000000.0,EOS,0.76,-6.79,-18.52,0.00066989,7.10769,10,EOS,900000000.0


In [26]:
coins_df['24h_volume_usd'] = pd.to_numeric(coins_df['24h_volume_usd'])
top_20 = coins_df.nlargest(20, '24h_volume_usd')['symbol']
top_20
#.sort_values(by=['24h_volume_usd'], ascending=False, inplace=True)

0      BTC
15    USDT
1      ETH
2      XRP
4      LTC
3      BCH
14     ETC
9      EOS
13     TRX
6      NEO
16     VEN
17    QTUM
5      ADA
11    DASH
8      XMR
36     WTC
22     ZEC
77     NXS
19    NANO
25     DGD
Name: symbol, dtype: object

In [32]:
import itertools
top_20_combinations = list(itertools.combinations(top_20, 2))
top_20_combinations

[('BTC', 'USDT'),
 ('BTC', 'ETH'),
 ('BTC', 'XRP'),
 ('BTC', 'LTC'),
 ('BTC', 'BCH'),
 ('BTC', 'ETC'),
 ('BTC', 'EOS'),
 ('BTC', 'TRX'),
 ('BTC', 'NEO'),
 ('BTC', 'VEN'),
 ('BTC', 'QTUM'),
 ('BTC', 'ADA'),
 ('BTC', 'DASH'),
 ('BTC', 'XMR'),
 ('BTC', 'WTC'),
 ('BTC', 'ZEC'),
 ('BTC', 'NXS'),
 ('BTC', 'NANO'),
 ('BTC', 'DGD'),
 ('USDT', 'ETH'),
 ('USDT', 'XRP'),
 ('USDT', 'LTC'),
 ('USDT', 'BCH'),
 ('USDT', 'ETC'),
 ('USDT', 'EOS'),
 ('USDT', 'TRX'),
 ('USDT', 'NEO'),
 ('USDT', 'VEN'),
 ('USDT', 'QTUM'),
 ('USDT', 'ADA'),
 ('USDT', 'DASH'),
 ('USDT', 'XMR'),
 ('USDT', 'WTC'),
 ('USDT', 'ZEC'),
 ('USDT', 'NXS'),
 ('USDT', 'NANO'),
 ('USDT', 'DGD'),
 ('ETH', 'XRP'),
 ('ETH', 'LTC'),
 ('ETH', 'BCH'),
 ('ETH', 'ETC'),
 ('ETH', 'EOS'),
 ('ETH', 'TRX'),
 ('ETH', 'NEO'),
 ('ETH', 'VEN'),
 ('ETH', 'QTUM'),
 ('ETH', 'ADA'),
 ('ETH', 'DASH'),
 ('ETH', 'XMR'),
 ('ETH', 'WTC'),
 ('ETH', 'ZEC'),
 ('ETH', 'NXS'),
 ('ETH', 'NANO'),
 ('ETH', 'DGD'),
 ('XRP', 'LTC'),
 ('XRP', 'BCH'),
 ('XRP', 'ETC'),
 ('

In [25]:
# candlestick
klines = bnb_client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_1MINUTE, "1 Jan, 2018", "2 Jan, 2018")
df = pd.DataFrame(klines, columns=['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume',
                             'Number of trades', 'Taker buy base asset volume','Taker buy quote asset volume','Ignore']
              )
df

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1514764800000,0.05358600,0.05361300,0.05349600,0.05352000,162.31200000,1514764859999,8.69187533,276,102.21300000,5.47571839,0
1,1514764860000,0.05352200,0.05359900,0.05352200,0.05358000,194.67100000,1514764919999,10.42554554,171,36.34700000,1.94703716,0
2,1514764920000,0.05358000,0.05358000,0.05347400,0.05357900,228.43200000,1514764979999,12.23152682,324,89.49700000,4.79315380,0
3,1514764980000,0.05357900,0.05359800,0.05351900,0.05352900,249.47800000,1514765039999,13.35954932,277,125.31700000,6.71312892,0
4,1514765040000,0.05355800,0.05364000,0.05353000,0.05362100,256.47400000,1514765099999,13.74583459,328,153.11500000,8.20809159,0
5,1514765100000,0.05357300,0.05365400,0.05356700,0.05358800,194.33300000,1514765159999,10.41596924,240,71.76000000,3.84822163,0
6,1514765160000,0.05358500,0.05368600,0.05358500,0.05360200,235.86000000,1514765219999,12.64941926,280,89.10000000,4.77970475,0
7,1514765220000,0.05360300,0.05369000,0.05359700,0.05363800,153.64300000,1514765279999,8.24212774,214,67.40100000,3.61743281,0
8,1514765280000,0.05363800,0.05369700,0.05358500,0.05369200,211.89100000,1514765339999,11.36771711,257,124.94400000,6.70567220,0
9,1514765340000,0.05369100,0.05372000,0.05361300,0.05371800,158.95900000,1514765399999,8.52916801,251,116.85700000,6.27134260,0


In [26]:
df.dtypes

Open time                        int64
Open                            object
High                            object
Low                             object
Close                           object
Volume                          object
Close time                       int64
Quote asset volume              object
Number of trades                 int64
Taker buy base asset volume     object
Taker buy quote asset volume    object
Ignore                          object
dtype: object

In [35]:
klines = bnb_client.get_historical_klines("TRXETH", Client.KLINE_INTERVAL_1MINUTE, '2 min ago UTC')
target_df = pd.DataFrame(klines, columns=['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume',
                             'Number of trades', 'Taker buy base asset volume','Taker buy quote asset volume','Ignore']
              )

klines = bnb_client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_1MINUTE, '2 min ago UTC')
alt_df = pd.DataFrame(klines, columns=['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume',
                             'Number of trades', 'Taker buy base asset volume','Taker buy quote asset volume','Ignore']
              )

klines = bnb_client.get_historical_klines("ETHUSDT", Client.KLINE_INTERVAL_1MINUTE, '2 min ago UTC')
through_df = pd.DataFrame(klines, columns=['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume',
                             'Number of trades', 'Taker buy base asset volume','Taker buy quote asset volume','Ignore']
              )

In [36]:
target_df

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1528521960000,0.00009695,0.00009696,0.00009675,0.00009696,79569.00000000,1528522019999,7.70929067,16,48243.00000000,4.67707952,0
1,1528522020000,0.00009687,0.00009687,0.00009687,0.00009687,159.00000000,1528522079999,0.01540233,1,0.00000000,0.00000000,0


In [37]:
alt_df

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1528521960000,0.07911500,0.07917200,0.07911500,0.07911700,65.52700000,1528522019999,5.18485510,82,29.56300000,2.33953507,0
1,1528522020000,0.07911500,0.07911700,0.07910100,0.07911400,40.48700000,1528522079999,3.20300986,39,19.99300000,1.58172085,0


In [44]:
through_df.sort_values('Open time').iloc[-1]

Open time                       1528522020000
Open                             605.27000000
High                             605.69000000
Low                              605.01000000
Close                            605.69000000
Volume                             1.29265000
Close time                      1528522079999
Quote asset volume               782.58802210
Number of trades                           12
Taker buy base asset volume        0.63654000
Taker buy quote asset volume     385.54591260
Ignore                                      0
Name: 1, dtype: object

In [ ]:
datetime.datetime.utcfromtimestamp(int(dateparser.parse(start_date).timestamp()/60/60/24)*24*60*60).strftime('%Y-%m-%d')

In [54]:
through_df['date'] = pd.to_datetime(through_df['Open time']/1000, unit='s')
through_df['date'].dt.hour

0    5
1    5
Name: date, dtype: int64

In [57]:
pd.to_datetime(through_df['Open time']/1000, unit='s').dt.dayofweek

0    5
1    5
Name: Open time, dtype: int64

In [23]:
# candlestick
import time
print(time.time())
klines = bnb_client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_1MINUTE, "1 minutes ago UTC")
pd.DataFrame(klines, columns=['Open time','Open','High','Low','Close','Volume','Close time','Quote asset volume',
                             'Number of trades', 'Taker buy base asset volume','Taker buy quote asset volume','Ignore']
              )

1528513326.4165142


,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1528513320000,0.07928400,0.07930000,0.07928400,0.07928500,4.42300000,1528513379999,0.35069306,14,2.13900000,0.16960618,0


In [4]:
prices = client.get_all_tickers()
prices

[{'price': '0.07448300', 'symbol': 'ETHBTC'},
 {'price': '0.01818000', 'symbol': 'LTCBTC'},
 {'price': '0.00091150', 'symbol': 'BNBBTC'},
 {'price': '0.01018200', 'symbol': 'NEOBTC'},
 {'price': '0.00030000', 'symbol': '123456'},
 {'price': '0.02986000', 'symbol': 'QTUMETH'},
 {'price': '0.00884400', 'symbol': 'EOSETH'},
 {'price': '0.00020869', 'symbol': 'SNTETH'},
 {'price': '0.00573600', 'symbol': 'BNTETH'},
 {'price': '0.11065000', 'symbol': 'BCCBTC'},
 {'price': '0.00302900', 'symbol': 'GASBTC'},
 {'price': '0.01223700', 'symbol': 'BNBETH'},
 {'price': '10599.51000000', 'symbol': 'BTCUSDT'},
 {'price': '790.00000000', 'symbol': 'ETHUSDT'},
 {'price': '0.00071800', 'symbol': 'HSRBTC'},
 {'price': '0.00107520', 'symbol': 'OAXETH'},
 {'price': '0.00010406', 'symbol': 'DNTETH'},
 {'price': '0.01071800', 'symbol': 'MCOETH'},
 {'price': '0.00258750', 'symbol': 'ICNETH'},
 {'price': '0.00079900', 'symbol': 'MCOBTC'},
 {'price': '0.00200000', 'symbol': 'WTCBTC'},
 {'price': '0.02690100', 

In [17]:
import pandas as pd
asks_df = pd.DataFrame(depth['asks'], columns=['price','volume','to_drop'])
asks_df['coin'] = 'BNBBTC'
asks_df.drop('to_drop', axis=1, inplace=True)
asks_df

,price,volume,coin
0,0.00090240,6.46000000,BNBBTC
1,0.00090270,551.72000000,BNBBTC
2,0.00090280,328.01000000,BNBBTC
3,0.00090340,24.00000000,BNBBTC
4,0.00090350,55.56000000,BNBBTC
5,0.00090380,1.77000000,BNBBTC
6,0.00090430,1.11000000,BNBBTC
7,0.00090440,9.00000000,BNBBTC
8,0.00090450,10.92000000,BNBBTC
9,0.00090500,75.10000000,BNBBTC


In [18]:
import json
json.dumps(depth)

'{"lastUpdateId": 38017581, "bids": [["0.00090210", "127.40000000", []], ["0.00090190", "7.08000000", []], ["0.00090170", "22.19000000", []], ["0.00090110", "6.66000000", []], ["0.00090100", "5.13000000", []], ["0.00090080", "41.84000000", []], ["0.00090070", "6.00000000", []], ["0.00090060", "90.85000000", []], ["0.00090050", "222.06000000", []], ["0.00090040", "12.00000000", []], ["0.00090020", "1.19000000", []], ["0.00090010", "28.13000000", []], ["0.00090000", "115.22000000", []], ["0.00089990", "200.00000000", []], ["0.00089960", "25.37000000", []], ["0.00089930", "43.02000000", []], ["0.00089900", "14.04000000", []], ["0.00089890", "23.32000000", []], ["0.00089870", "3.35000000", []], ["0.00089850", "9.24000000", []], ["0.00089820", "33.12000000", []], ["0.00089810", "2.93000000", []], ["0.00089800", "10.97000000", []], ["0.00089770", "6.00000000", []], ["0.00089750", "52.00000000", []], ["0.00089710", "12.00000000", []], ["0.00089700", "5.00000000", []], ["0.00089690", "23.04000

In [56]:
import pandas as pd
recents = client.get_recent_trades(symbol='BNBBTC',)
recents_df = pd.DataFrame(recents)
recents_df.time.iloc[0]

1521295718080

In [18]:
recents_df[recents_df['isBestMatch'] == False]

,id,isBestMatch,isBuyerMaker,price,qty,time


In [20]:
# Seconds between
(recents_df.time.iloc[-1] - recents_df.time.iloc[0]) / 1000

379.85000000000002

In [22]:
# Create market maker metric to embody the price/timing/qty movement of trades 

In [88]:
prices = client.get_all_tickers()
prices

[{'price': '0.08080000', 'symbol': 'ETHBTC'},
 {'price': '0.02003200', 'symbol': 'LTCBTC'},
 {'price': '0.00090260', 'symbol': 'BNBBTC'},
 {'price': '0.01138000', 'symbol': 'NEOBTC'},
 {'price': '0.00030000', 'symbol': '123456'},
 {'price': '0.03195100', 'symbol': 'QTUMETH'},
 {'price': '0.00986100', 'symbol': 'EOSETH'},
 {'price': '0.00021356', 'symbol': 'SNTETH'},
 {'price': '0.00587200', 'symbol': 'BNTETH'},
 {'price': '0.12524600', 'symbol': 'BCCBTC'},
 {'price': '0.00400000', 'symbol': 'GASBTC'},
 {'price': '0.01116000', 'symbol': 'BNBETH'},
 {'price': '10939.89000000', 'symbol': 'BTCUSDT'},
 {'price': '884.69000000', 'symbol': 'ETHUSDT'},
 {'price': '0.00103900', 'symbol': 'HSRBTC'},
 {'price': '0.00092010', 'symbol': 'OAXETH'},
 {'price': '0.00010113', 'symbol': 'DNTETH'},
 {'price': '0.00914900', 'symbol': 'MCOETH'},
 {'price': '0.00173360', 'symbol': 'ICNETH'},
 {'price': '0.00073600', 'symbol': 'MCOBTC'},
 {'price': '0.00196000', 'symbol': 'WTCBTC'},
 {'price': '0.02426800', 

### Orders data

In [8]:
trades = bnb_client.get_my_trades(symbol='ETHUSDT')
trades

[{'id': 21647347,
  'orderId': 58980955,
  'price': '601.30000000',
  'qty': '0.42749000',
  'commission': '0.25704974',
  'commissionAsset': 'USDT',
  'time': 1524364393820,
  'isBuyer': False,
  'isMaker': False,
  'isBestMatch': True},
 {'id': 21647497,
  'orderId': 58981419,
  'price': '601.95000000',
  'qty': '1.28247000',
  'commission': '0.77198282',
  'commissionAsset': 'USDT',
  'time': 1524364500388,
  'isBuyer': False,
  'isMaker': True,
  'isBestMatch': True},
 {'id': 21648572,
  'orderId': 58985563,
  'price': '604.12000000',
  'qty': '0.44000000',
  'commission': '0.00044000',
  'commissionAsset': 'ETH',
  'time': 1524365274591,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'id': 21648573,
  'orderId': 58985563,
  'price': '604.20000000',
  'qty': '0.44000000',
  'commission': '0.00044000',
  'commissionAsset': 'ETH',
  'time': 1524365274591,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True},
 {'id': 21648574,
  'orderId': 58985563,
  'price

In [9]:
orders = bnb_client.get_all_orders(symbol='ETHUSDT', limit=10)
orders

[{'symbol': 'ETHUSDT',
  'orderId': 88262311,
  'clientOrderId': 'zS1RfDHIsqQp70uYPwAW01',
  'price': '0.00000000',
  'origQty': '1.00000000',
  'executedQty': '1.00000000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1530876689225,
  'isWorking': True},
 {'symbol': 'ETHUSDT',
  'orderId': 88266847,
  'clientOrderId': '1meZxD09iOSBDMqKtv0wer',
  'price': '0.00000000',
  'origQty': '1.00000000',
  'executedQty': '1.00000000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1530877890476,
  'isWorking': True},
 {'symbol': 'ETHUSDT',
  'orderId': 88270369,
  'clientOrderId': 'KMCR697lNstDhzcbqHgnlG',
  'price': '0.00000000',
  'origQty': '1.00000000',
  'executedQty': '1.00000000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '

In [7]:
# Dataframe
order_list = list(orders)
order_df = pd.DataFrame(order_list)
order_df

,clientOrderId,executedQty,icebergQty,isWorking,orderId,origQty,price,side,status,stopPrice,symbol,time,timeInForce,type
0,ASB6deksGntTdHK8pj4AHt,1.00000000,0.00000000,True,88232836,1.00000000,0.00000000,BUY,FILLED,0.00000000,ETHUSDT,1530869653265,GTC,MARKET
1,aQXK1ckzhAzGooqAp5vN2H,1.00000000,0.00000000,True,88237384,1.00000000,0.00000000,SELL,FILLED,0.00000000,ETHUSDT,1530870853515,GTC,MARKET
2,fAhJrtzL7U80ZGeXpVLEjI,1.00000000,0.00000000,True,88251552,1.00000000,0.00000000,BUY,FILLED,0.00000000,ETHUSDT,1530874020371,GTC,MARKET
3,0kzVX1chXSpe7Iy6hU7bod,1.00000000,0.00000000,True,88257585,1.00000000,0.00000000,SELL,FILLED,0.00000000,ETHUSDT,1530875220644,GTC,MARKET
4,zS1RfDHIsqQp70uYPwAW01,1.00000000,0.00000000,True,88262311,1.00000000,0.00000000,BUY,FILLED,0.00000000,ETHUSDT,1530876689225,GTC,MARKET
5,1meZxD09iOSBDMqKtv0wer,1.00000000,0.00000000,True,88266847,1.00000000,0.00000000,SELL,FILLED,0.00000000,ETHUSDT,1530877890476,GTC,MARKET
6,KMCR697lNstDhzcbqHgnlG,1.00000000,0.00000000,True,88270369,1.00000000,0.00000000,BUY,FILLED,0.00000000,ETHUSDT,1530878811219,GTC,MARKET
7,3Zbj3YeibfYEDXtgH90wKC,1.00000000,0.00000000,True,88279304,1.00000000,0.00000000,SELL,FILLED,0.00000000,ETHUSDT,1530880011639,GTC,MARKET
8,ZjVwtajItAaMWOuW4213TX,1.00000000,0.00000000,True,88285198,1.00000000,0.00000000,BUY,FILLED,0.00000000,ETHUSDT,1530881271503,GTC,MARKET
9,tkQ7KdNF6rH919MVfoSBcR,1.00000000,0.00000000,True,88291654,1.00000000,0.00000000,SELL,FILLED,0.00000000,ETHUSDT,1530882471734,GTC,MARKET


In [ ]:
df.rename(columns={'a':'trade_id','p':'price','q':'quantity','f':'first_trade_id','l':'last_trade_id',
                   'T':'micro_timestamp','m':'buyer_maker','M':'best_price_match'}, inplace=True)
df['unix_timestamp'] = df['micro_timestamp']/1000
df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='s')
df = df[['trade_id','datetime','price','quantity','buyer_maker','best_price_match',
         'first_trade_id','last_trade_id','micro_timestamp']]
df